In [4]:
import numpy as np
from scipy import sparse 
from sklearn.metrics import precision_score, recall_score, f1_score

In [5]:
N = 15000 # Number of samples
d = 16 # Features
C = 26 # Class numbers

In [6]:
# Function to get data from files
def read_data(file_path):
    X, Y = [], []
    with open(file_path, 'r') as file:
        for line in file:
            data = line.strip().split(' ')
            label = int(data[0])  
            features = [float(feature.split(':')[1]) for feature in data[1:]]  
            X.append(features)
            Y.append(label)
    return np.array(X), np.array(Y)

### Loading data

In [7]:
file_train_path = "./letter.scale.tr" # Training dataset
file_test_path = "./letter.scale.t" # Testing dataset
file_val_path = "./letter.scale.val" # Validation dataset

# Loading training dataset
X_train, y_train = read_data(file_train_path)
X_train = X_train.T
print(X_train.shape)
print(y_train.shape)

# Loading testing dataset
X_test, y_test = read_data(file_test_path)
X_test = X_test.T
print(X_test.shape)
print(y_test.shape)

# Loading validation dataset
X_val, y_val= read_data(file_val_path)
X_val = X_val.T
print(X_val.shape)
print(y_val.shape)

(16, 10500)
(10500,)
(16, 5000)
(5000,)
(16, 4500)
(4500,)


In [31]:
X_train.shape[1]

10500

In [8]:
# Origin label is from 1 -> 26, so we need to subtract all by 1 to make sure that it goes from 0 -> 25
y_train = y_train - 1
y_test = y_test - 1
y_val = y_val - 1 

In [37]:
def one_hot_encoding(y, num_classes):

    Y = sparse.coo_matrix((np.ones_like(y), 
        (y, np.arange(len(y)))), shape=(num_classes, len(y))).toarray()
    return Y 


def cross_entropy_loss(X, Y_hat, theta): # Y_hat is onehot form
    Y = softmax_stable(np.dot(theta.T, X))
    eps = 1e-15
    Y_hat = np.clip(Y_hat, eps, 1 - eps)
    loss = -np.sum(Y * np.log(Y_hat)) / Y.shape[1]
    return loss 

def softmax(Z):
    e_Z = np.exp(Z)
    A = e_Z / e_Z.sum(axis=0)
    return A

def softmax_stable(Z):
    e_Z = np.exp(Z - np.max(Z, axis=0, keepdims=True))
    A = e_Z / e_Z.sum(axis=0)
    return A


In [19]:
def accuracy_score(y_true, y_pred):
    correct_predictions = np.sum(y_true == y_pred)
    total_predictions = len(y_true)
    accuracy = (correct_predictions / total_predictions) * 100
    return accuracy

def predict(theta, X):
    A = softmax_stable(np.dot(theta.T, X))
    return np.argmax(A, axis=0)

def calculate_f1_score(y_true, y_pred):
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    return precision, recall, f1



#### Constant step size

In [20]:
def softmax_regression(X, y, theta_init, eta, tol=1e-8, max_count=500000):
    theta = theta_init
    C = theta.shape[1]
    Y = one_hot_encoding(y, C)
    N = X.shape[1]
    d = X.shape[0]
    
    count = 0
    prev_theta = theta

    while count < max_count:
        # Shuffle data
        mix_id = np.random.permutation(N)
        for i in mix_id:
            xi = X[:, i].reshape(d, 1)
            yi = Y[:, i].reshape(C, 1)
            ai = softmax(np.dot(theta.T, xi))
            grad = xi.dot((ai - yi).T)
            theta_new = theta - eta * grad
            count += 1
            
            # Check for convergence
            if np.linalg.norm(theta_new - prev_theta) < tol:
                return theta_new
            
            prev_theta = theta
            theta = theta_new
    
    return theta

#### Backtracking

In [34]:
def backtraking_softmax(X_train, y_train, eta=1, m=0.5, alpha=0.5, batch_size=1024, epochs=256):
    y_train1hot = one_hot_encoding(y_train, 26)
    d, C = X_train.shape[0], y_train1hot.shape[0]
    theta_init = np.random.randn(d, C)
    theta = [theta_init]

    for epoch in range(epochs):
        for i in range(0, X_train.shape[1], batch_size):
            # Get the batch
            X_batch = X_train[:, i:i+batch_size]
            y_batch = y_train1hot[:, i:i+batch_size]

            # Compute the gradient over the batch
            ai = softmax_stable(np.dot(theta[-1].T, X_batch))
            grad = np.dot(X_batch, (ai - y_batch).T) / batch_size

            # Update theta with backtracking line search
            theta_new = theta[-1] - eta * grad
            t_k = eta
            j = 0
            while (cross_entropy_loss(X_train, y_train1hot, theta_new) > cross_entropy_loss(X_train, y_train1hot, theta[-1]) - m * t_k * np.linalg.norm(grad) ** 2) and (j < 5):
                t_k *= alpha
                theta_new = theta[-1] - t_k * grad
                j += 1

            # Append the new theta to the list
            theta.append(theta_new)
            if np.linalg.norm(theta[-1] - theta[-2]) < 1e-8:
                print(f"Converged at epoch {epoch + 1}, iteration {i // batch_size + 1}")
                return theta[-1]

        print(f"Epoch {epoch + 1}, Loss: {cross_entropy_loss(X_train, y_train1hot, theta[-1])}")

    # Final trained theta
    return theta[-1]

# Training models

## Constant step-size model

In [22]:
eta = 0.03
theta_init = np.random.randn(d, C)
theta = softmax_regression(X_train, y_train, theta_init, eta)

In [23]:
print(theta)

[[  1.64431722  -4.1577112    1.56590038  -7.0136261   -3.99226107
   -2.69107225   0.11563527  -4.34672849   3.87485429   0.39681656
   -0.05464407   0.40487192   2.02555134  -1.27590916  -2.24071501
    1.13816917   3.08357054  -1.95806934  -0.10107353   2.14360767
    0.13745665   1.80319592   4.40600431  -0.78687476   3.65020917
   -1.21479458]
 [ -2.69535216   2.61164918   1.46747786   5.31158617   1.37125232
   -0.70923241   0.767559     1.17144991  -0.58689666  -3.28040068
   -1.22775965   0.79344677  -0.28171723   0.11502387   2.10153485
   -3.03091997  -2.36813771   1.01228289  -0.61013629   1.66164031
    2.74274373   0.7637991   -2.06121909  -1.01795518  -2.33841057
   -3.9693405 ]
 [  2.34817122  -2.03906091  -2.46862142  -2.08908117  -2.92717831
    0.25921721   2.11992192   9.65190989 -11.5719076   -3.98175953
    5.00953413  -3.69424656   6.6795214    6.60473442   2.2327069
    0.11339362  -1.69991474  -0.27961241   1.62333231  -5.13491295
   -1.03025006   1.33921553   1

### Prediction on constant step-size model

In [24]:
# Predict on the training set
Y_train_pred = predict(theta, X_train)  
print("Predicted classes for the first 20 data points in the training set:\n", Y_train_pred[:20])
print("Actual classes for the first 20 data points in the training set:\n", y_train[:20])
acc_train = accuracy_score(y_train.T, Y_train_pred)
print(f"Accuracy for the training set: {acc_train:.2f}%")

# Calculate Precision, Recall, and F1 score on the training set
precision, recall, f1 = calculate_f1_score(y_train, Y_train_pred)

print(f'Weighted Precision on the training set: {precision:.2f}')
print(f'Weighted Recall on the training set: {recall:.2f}')
print(f'Weighted F1 score on the training set: {f1:.2f}')


Predicted classes for the first 20 data points in the training set:
 [21 11  8  2 21 10  9  3  7  6  9  4  3  2 21 18 24 11 23 19]
Actual classes for the first 20 data points in the training set:
 [21 11  8  2 21 10  9  3  7  6  9  2  3  6 21  0 24 11 23 19]
Accuracy for the training set: 75.67%
Weighted Precision on the training set: 0.75
Weighted Recall on the training set: 0.76
Weighted F1 score on the training set: 0.75


In [25]:
# Predict on the test set
Y_test_pred = predict(theta, X_test)
print("Predicted classes for the first 20 data points in the test set:\n", Y_test_pred[:20])
print("Actual classes for the first 20 data points in the test set:\n", y_test[:20])

acc_test = accuracy_score(y_test.T, Y_test_pred)
print(f"Accuracy for the test set: {acc_test:.2f}%")

# Calculate Precision, Recall, and F1 score on the test set
test_precision, test_recall, test_f1 = calculate_f1_score(y_test, Y_test_pred)

print(f'Weighted Precision on the test set: {test_precision:.2f}')
print(f'Weighted Recall on the test set: {test_recall:.2f}')
print(f'Weighted F1 score on the test set: {test_f1:.2f}')


Predicted classes for the first 20 data points in the test set:
 [16 17  5  5 19 19 16  9  2 14 21  8 14 17  4 16  5 23  5  4]
Actual classes for the first 20 data points in the test set:
 [16 17  5  5 19 19  6 25  2 14 21  8  7  3  4 16 15 23  5  4]
Accuracy for the test set: 75.12%
Weighted Precision on the test set: 0.75
Weighted Recall on the test set: 0.75
Weighted F1 score on the test set: 0.75


In [26]:
# Predict on the validation set
Y_val_pred = predict(theta, X_val)
print("Predicted classes for the first 20 data points in the validation set:\n", Y_val_pred[:20])
print("Actual classes for the first 20 data points in the validation set:\n", y_val[:20])

acc_val = accuracy_score(y_val.T, Y_val_pred)
print(f"Accuracy for the validation set: {acc_val:.2f}%")

# Calculate Precision, Recall, and F1 score on the validation set
val_precision, val_recall, val_f1 = calculate_f1_score(y_val, Y_val_pred)

print(f'Weighted Precision on the validation set: {val_precision:.2f}')
print(f'Weighted Recall on the validation set: {val_recall:.2f}')
print(f'Weighted F1 score on the validation set: {val_f1:.2f}')


Predicted classes for the first 20 data points in the validation set:
 [13  5  2 10  9 11  0  1 17  4 25  1 15 11 13 17  5 12  4 19]
Actual classes for the first 20 data points in the validation set:
 [13  5  6 17  9 11  0  1 18 25 18  1 15 11 13 17  5 12  4 19]
Accuracy for the validation set: 75.40%
Weighted Precision on the validation set: 0.75
Weighted Recall on the validation set: 0.75
Weighted F1 score on the validation set: 0.75


## Backtracking (Armijo step size)

In [38]:
backtracking_theta = backtraking_softmax(X_train, y_train, eta=1, m=0.5, alpha=0.5, batch_size=1024, epochs=256)

Epoch 1, Loss: 33.14231549145548
Epoch 2, Loss: 33.115271150662984
Epoch 3, Loss: 33.046731851486705
Epoch 4, Loss: 32.7040069160328
Epoch 5, Loss: 32.43478574341474
Epoch 6, Loss: 32.1807675998398
Epoch 7, Loss: 31.921270556515417
Epoch 8, Loss: 31.6556046072193
Epoch 9, Loss: 31.38439749795307
Epoch 10, Loss: 31.108175989858907
Epoch 11, Loss: 30.828196023612723
Epoch 12, Loss: 30.546467013653555
Epoch 13, Loss: 30.265409342264647
Epoch 14, Loss: 29.987457683052863
Epoch 15, Loss: 29.71472931937366
Epoch 16, Loss: 29.44882492250892
Epoch 17, Loss: 29.190778150648963
Epoch 18, Loss: 28.941117134010874
Epoch 19, Loss: 28.699979908152734
Epoch 20, Loss: 28.46723663037537
Epoch 21, Loss: 28.2425931307137
Epoch 22, Loss: 28.025667948829188
Epoch 23, Loss: 27.81604462013765
Epoch 24, Loss: 27.613304504623365
Epoch 25, Loss: 27.417045745434745
Epoch 26, Loss: 27.22689298005967
Epoch 27, Loss: 27.04250121577018
Epoch 28, Loss: 26.86355622288597
Epoch 29, Loss: 26.68977299487393
Epoch 30, Los

### Prediction on backtracking step-size model

In [39]:
# Predict on the training set with the backtracking algorithm
Y_train_pred = predict(backtracking_theta, X_train)
print("Predicted classes for the first 30 data points in the training set with the backtracking algorithm:\n", Y_train_pred[:30])
print("Actual classes for the first 30 data points in the training set with the backtracking algorithm:\n", y_train[:30])

acc_train = accuracy_score(y_train.T, Y_train_pred)
print(f"Accuracy for the training set with the backtracking algorithm: {acc_train:.2f}%")

# Calculate Precision, Recall, and F1 score on the training set with the backtracking algorithm
precision, recall, f1 = calculate_f1_score(y_train, Y_train_pred)

print(f'Weighted Precision on the training set with the backtracking algorithm: {precision:.2f}')
print(f'Weighted Recall on the training set with the backtracking algorithm: {recall:.2f}')
print(f'Weighted F1 score on the training set with the backtracking algorithm: {f1:.2f}')


Predicted classes for the first 30 data points in the training set with the backtracking algorithm:
 [21 11  8  2 21 16  9  3  7 10  9  4  3  2 21 18 24 11 23 19 24 16 12 16
 20 21 13 21 17 13]
Actual classes for the first 30 data points in the training set with the backtracking algorithm:
 [21 11  8  2 21 10  9  3  7  6  9  2  3  6 21  0 24 11 23 19 24  6 12 16
 20 21 20 24 23 13]
Accuracy for the training set with the backtracking algorithm: 72.18%
Weighted Precision on the training set with the backtracking algorithm: 0.72
Weighted Recall on the training set with the backtracking algorithm: 0.72
Weighted F1 score on the training set with the backtracking algorithm: 0.72


In [40]:
# Predict on the test set with the backtracking algorithm
Y_test_pred = predict(backtracking_theta, X_test)
print("Predicted classes for the first 20 data points in the test set with the backtracking algorithm:\n", Y_test_pred[:20])
print("Actual classes for the first 20 data points in the test set with the backtracking algorithm:\n", y_test[:20])
acc_test = accuracy_score(y_test.T, Y_test_pred)
print(f"Accuracy for the test set with the backtracking algorithm: {acc_test:.2f}%")

# Calculate Precision, Recall, and F1 score on the test set with the backtracking algorithm
test_precision, test_recall, test_f1 = calculate_f1_score(y_test, Y_test_pred)

print(f'Weighted Precision on the test set with the backtracking algorithm: {test_precision:.2f}')
print(f'Weighted Recall on the test set with the backtracking algorithm: {test_recall:.2f}')
print(f'Weighted F1 score on the test set with the backtracking algorithm: {test_f1:.2f}')


Predicted classes for the first 20 data points in the test set with the backtracking algorithm:
 [14 17  5  5 19 19  1  5  2 14 21  8  7 17 11  1  5 23  5  4]
Actual classes for the first 20 data points in the test set with the backtracking algorithm:
 [16 17  5  5 19 19  6 25  2 14 21  8  7  3  4 16 15 23  5  4]
Accuracy for the test set with the backtracking algorithm: 71.98%
Weighted Precision on the test set with the backtracking algorithm: 0.72
Weighted Recall on the test set with the backtracking algorithm: 0.72
Weighted F1 score on the test set with the backtracking algorithm: 0.71


In [41]:
# Predict on the validation set with the backtracking algorithm
Y_val_pred = predict(backtracking_theta, X_val)
print("Predicted classes for the first 20 data points in the validation set with the backtracking algorithm:\n", Y_val_pred[:20])
print("Actual classes for the first 20 data points in the validation set with the backtracking algorithm:\n", y_val[:20])

acc_val = accuracy_score(y_val.T, Y_val_pred)
print(f"Accuracy for the validation set with the backtracking algorithm: {acc_val:.2f}%")

# Calculate Precision, Recall, and F1 score on the validation set with the backtracking algorithm
val_precision, val_recall, val_f1 = calculate_f1_score(y_val, Y_val_pred)

print(f'Weighted Precision on the validation set with the backtracking algorithm: {val_precision:.2f}')
print(f'Weighted Recall on the validation set with the backtracking algorithm: {val_recall:.2f}')
print(f'Weighted F1 score on the validation set with the backtracking algorithm: {val_f1:.2f}')


Predicted classes for the first 20 data points in the validation set with the backtracking algorithm:
 [13  5  2 17  9 11  0  1 17 25 18  1 15 11 13 17  5 12  4 19]
Actual classes for the first 20 data points in the validation set with the backtracking algorithm:
 [13  5  6 17  9 11  0  1 18 25 18  1 15 11 13 17  5 12  4 19]
Accuracy for the validation set with the backtracking algorithm: 71.89%
Weighted Precision on the validation set with the backtracking algorithm: 0.72
Weighted Recall on the validation set with the backtracking algorithm: 0.72
Weighted F1 score on the validation set with the backtracking algorithm: 0.71
